our goal

have each .dat file treated as a single row with 2000 data points (will be resized to 50 x 40)
put each one of these .dat files into a main dataframe
i.e.

(a+c+t+g # of dat, 300) x (2000)

each single datapoint will be resized to a picture, 50x40, we have 300 of them
75 pictures per base

make a labels file, 300 x 4 (one hot encode)

In [75]:
import sys
import os
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torch.nn.functional as F

In [262]:
data = np.zeros((297,2000)) #since _A58 had 1224 points, A_68 also had less substract by 2?
i = a_count = c_count = g_count = t_count = 0
source_dir = Path('DNA/')

for folders in os.listdir("DNA/"):
    print(folders)
    source_dir = Path('DNA/'+folders)
    # absolute_path = os.path.abspath(source_dir)
    # print("Full path: " + absolute_path)
    # print("Directory Path: " + os.path.dirname(absolute_path))

    for raw_data in os.listdir(source_dir):
        data_path = Path('DNA/'+folders+'/'+raw_data)
        # print(i, raw_data, data_path)

        # absolute_path = os.path.abspath(data_path)
        # print("Full path: " + absolute_path)
        # print("Directory Path: " + os.path.dirname(absolute_path))
        # print(np.genfromtxt(data_path, delimiter='\t'))
        # print(np.loadtxt(data_path))
        # print(pd.read_csv(data_path,sep=' ',names = ['time', 'current']))

        data[i,:] = pd.read_csv(data_path,sep=' ',names = ['time', 'current'])['current'][0:2000]
        # print(i, data[i,:])
        
        i += 1
        if folders == 'A':
            a_count += 1
        elif folders == 'C':
            c_count += 1
        elif folders == 'G':
            g_count += 1
        else:
            t_count += 1


print(data)
print(a_count, c_count, g_count, t_count)

labels = np.zeros((297,4))
for j in range(len(labels)):
    if j < a_count:
        labels[j,0] = 1
    elif a_count <= j < (a_count + c_count):
        labels[j,1] = 1
    elif (a_count + c_count) <= j < (a_count + c_count + g_count):
        labels[j,2] = 1
    elif j >= (a_count + c_count + g_count):
        labels[j,3] = 1

print(labels, np.sum(labels[:,0]), np.sum(labels[:,1]), np.sum(labels[:,2]), np.sum(labels[:,3]))

labels2 = np.zeros((297))
for j in range(len(labels)):
    if j < a_count:
        labels2[j] = 0
    elif a_count <= j < (a_count + c_count):
        labels2[j] = 1
    elif (a_count + c_count) <= j < (a_count + c_count + g_count):
        labels2[j] = 2
    elif j >= (a_count + c_count + g_count):
        labels2[j] = 3

A
C
G
T
[[-207.97638578  180.07280633  124.81725379 ... -437.1953854
   206.75177055  -98.57610457]
 [ -67.90753058 -139.75267189 -123.63092518 ...  219.0008219
  -126.9087813  -133.07615075]
 [-200.57816597   74.44650396   23.99452835 ...  113.68002792
  -379.79792376   88.68582645]
 ...
 [ -27.06981436  117.20208853  271.27015217 ...  291.07867222
  -118.45682056   10.35573719]
 [  30.32838415  248.40394     -62.00122146 ... -193.95503378
   217.67315365 -257.97382595]
 [  10.79077708 -353.25044902  129.41285985 ...   94.67509104
   163.16901564 -129.20393461]]
73 75 74 75
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]] 73.0 75.0 74.0 75.0


In [263]:
# x=data[:,0:1600]
x=data[0:-1,200:-200]
labels=labels[0:-1,:]
labels2=labels2[0:-1]


print(x.shape, labels2.shape)

(296, 1600) (296,)


In [264]:
def mean(data,no_elements):
    X=np.zeros((data.shape[0],data.shape[1]))
    for i in range(data.shape[1]-no_elements+1):
        X[:,i]=np.mean(data[:,i:i+no_elements],axis=1)
    return X.astype(np.float16)
def median(data,no_elements):
    X=np.zeros((data.shape[0],data.shape[1]))
    for i in range(data.shape[1]-no_elements+1):
        X[:,i]=np.median(data[:,i:i+no_elements],axis=1)
    return X.astype(np.float16)
def sig_image(data,size1,size2):
    X=np.zeros((data.shape[0],size1,size2))
    for i in range(data.shape[0]):
        X[i]=(data[i,:].reshape(size1,size2))
    return X.astype(np.float16)

In [265]:
channel_mean=(mean(x,10)).astype(np.float16)
x_m=sig_image(channel_mean,40,40)
channel_median=(median(x,10)).astype(np.float16)
x_md=sig_image(x,40,40)

In [266]:
x_n=sig_image(x,40,40)

In [267]:
x_n.shape

(296, 40, 40)

In [268]:
x_m.shape

(296, 40, 40)

In [269]:
X=np.stack((x_n,x_m,x_md),axis=1).astype(np.float16)

In [270]:
X.shape

(296, 3, 40, 40)

In [271]:
from sklearn.model_selection import train_test_split
trainx, testx, trainlabel, testlabel = train_test_split(X, labels2, test_size=0.2, random_state=20)

# print(trainx.shape, testx.shape, trainlabel.shape, testlabel.shape)

(236, 3, 40, 40) (60, 3, 40, 40) (236,) (60,)


In [272]:
sig_train, sig_test = trainx,testx
lab_train, lab_test = trainlabel,testlabel

In [273]:
sig_train = torch.from_numpy(sig_train)
sig_test = torch.from_numpy(sig_test)
lab_train= torch.from_numpy(lab_train)
lab_test = torch.from_numpy(lab_test)

In [274]:
import torch.utils.data as data_utils
batch_size = 118
train_tensor = data_utils.TensorDataset(sig_train, lab_train) 
train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)

In [275]:
batch_size = 60
test_tensor = data_utils.TensorDataset(sig_test, lab_test) 
test_loader = data_utils.DataLoader(dataset = test_tensor, batch_size = batch_size, shuffle = False)

In [276]:
sig_train.size()

torch.Size([236, 3, 40, 40])

In [277]:
sig_test.size()

torch.Size([60, 3, 40, 40])

In [297]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=4,stride=1,padding = 1)
        self.mp1 = nn.MaxPool2d(kernel_size=4,stride=2)
        self.conv2 = nn.Conv2d(32,64, kernel_size=4,stride =1)
        self.mp2 = nn.MaxPool2d(kernel_size=4,stride=2)
        self.fc1= nn.Linear(2304,256)
        self.dp1 = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(256,4)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp1(self.conv1(x)))
        x = F.relu(self.mp2(self.conv2(x)))
        x = x.view(in_size,-1)
        x = F.relu(self.fc1(x))
        x = self.dp1(x)
        x = self.fc2(x)     
        print('here')   
        return F.log_softmax(x, dim=1)

In [279]:
cnn = CNN().double()

In [280]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)

In [281]:
num_epochs = 100

In [292]:
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (signals, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        # Run the forward pass
        signals=signals
        labels=labels
        outputs = cnn(signals.double())
        loss = criterion(outputs, labels.long())
        
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        
        loss.backward()
        optimizer.step()
        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels.long()).sum().item()
        acc_list.append(correct / total)

        if (epoch+1) % 5 == 0 or epoch==0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Train Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

torch.Size([118, 4])
Epoch [1/100], Step [1/2], Loss: nan, Train Accuracy: 25.42%
torch.Size([118, 4])
Epoch [1/100], Step [2/2], Loss: nan, Train Accuracy: 22.03%
torch.Size([118, 4])
torch.Size([118, 4])
torch.Size([118, 4])
torch.Size([118, 4])
torch.Size([118, 4])
torch.Size([118, 4])
torch.Size([118, 4])
Epoch [5/100], Step [1/2], Loss: nan, Train Accuracy: 25.42%
torch.Size([118, 4])
Epoch [5/100], Step [2/2], Loss: nan, Train Accuracy: 22.03%
torch.Size([118, 4])
torch.Size([118, 4])
torch.Size([118, 4])
torch.Size([118, 4])
torch.Size([118, 4])
torch.Size([118, 4])
torch.Size([118, 4])
torch.Size([118, 4])
torch.Size([118, 4])
Epoch [10/100], Step [1/2], Loss: nan, Train Accuracy: 27.12%
torch.Size([118, 4])
Epoch [10/100], Step [2/2], Loss: nan, Train Accuracy: 20.34%
torch.Size([118, 4])
torch.Size([118, 4])
torch.Size([118, 4])
torch.Size([118, 4])
torch.Size([118, 4])
torch.Size([118, 4])
torch.Size([118, 4])
torch.Size([118, 4])
torch.Size([118, 4])
Epoch [15/100], Step [1

In [295]:
total_step = len(test_loader)
print(total_step)
loss_list_test = []
acc_list_test = []
with torch.no_grad():
    for i, (signals, labels) in enumerate(test_loader):
        # Run the forward pass
        signals=signals
        labels=labels
        outputs = cnn(signals.double())
        loss = criterion(outputs, labels.long())
        loss_list_test.append(loss.item())
        if epoch%10 ==0:
            print(loss)
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels.long()).sum().item()
        acc_list_test.append(correct / total)
        if (epoch) % 1 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

1
torch.Size([60, 4])
Epoch [100/100], Step [1/1], Loss: nan, Accuracy: 28.33%


In [26]:
# if you need to save
torch.save(cnn,'cnnTC3_fold3_45.pth')

/home/cmu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
